In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, MaxPool2D

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# train_images = np.load('drive/MyDrive/Colab Notebooks/train_images_200.npy')
# train_labels = np.load('drive/MyDrive/Colab Notebooks/train_labels_200.npy')
test_images = np.load('drive/MyDrive/Colab Notebooks/test_images_200.npy')
test_labels = np.load('drive/MyDrive/Colab Notebooks/test_labels_200.npy')

In [4]:
from sklearn.model_selection import train_test_split

# train_images = np.array(train_images).reshape(-1, 96, 96, 3) / 255.0
test_images = np.array(test_images).reshape(-1, 96, 96, 3) / 255.0

train_images, test_images, train_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.2)

train_labels_one_hot = np.eye(2)[train_labels]
test_labels_one_hot = np.eye(2)[test_labels]

In [5]:
model = Sequential()

# Convolutional Block (Conv-Conv-Pool-Dropout)
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(96, 96, 3)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Convolutional Block (Conv-Conv-Pool-Dropout)
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) # change this line
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [6]:
import keras

def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 3:
        lrate = 0.0005
    if epoch > 5:
        lrate = 0.0001
    if epoch > 10:
        lrate = 0.00005

    return lrate

checkpoint = keras.callbacks.ModelCheckpoint('best_cnn_model.h5', save_best_only=True)
# N번의 연속적인 epoch 동안 개선이 없을 때 학습이 중단되도록 함
early_stopping = keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

In [7]:
from keras.callbacks import LearningRateScheduler

history = model.fit(train_images, train_labels_one_hot, batch_size=32, epochs=100, validation_data=(test_images, test_labels_one_hot),
                    callbacks=[checkpoint, early_stopping, LearningRateScheduler(lr_schedule)])

Epoch 1/100
250/250 [==============================] - 36s 89ms/step - loss: 2.4730 - acc: 0.5063 - val_loss: 0.6927 - val_acc: 0.5140 - lr: 0.0010
Epoch 2/100
250/250 [==============================] - 21s 83ms/step - loss: 0.6921 - acc: 0.5171 - val_loss: 0.6916 - val_acc: 0.5280 - lr: 0.0010
Epoch 3/100
250/250 [==============================] - 20s 81ms/step - loss: 0.6903 - acc: 0.5184 - val_loss: 0.6922 - val_acc: 0.5065 - lr: 0.0010
Epoch 4/100
250/250 [==============================] - 20s 80ms/step - loss: 0.6917 - acc: 0.5166 - val_loss: 0.6918 - val_acc: 0.5335 - lr: 0.0010
Epoch 5/100
135/250 [===============>..............] - ETA: 8s - loss: 0.6901 - acc: 0.5285

KeyboardInterrupt: ignored

In [ ]:
from keras.models import load_model

model = load_model('best_cnn_model.h5')

In [ ]:
import matplotlib.pyplot as plt

plt.figure()

# 정확도 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['acc'], label='acc')
plt.plot(history.history['val_acc'], label='val_acc')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(loc='best')

# 손실 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc='best')

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels_one_hot, verbose=2)

In [ ]:
# 테스트 데이터셋에 대한 예측 생성
pred_probs = model.predict(test_images[:100])

print(pred_probs)